### try real world data 

In [ ]:
import logging
import os
import pickle
import random
import sys

# from bds.bbSampleTree import BranchAndBoundNaive
# from bds.bbSampleTree import BranchAndBoundNaive
import time
from time import time

import gmpy2 as gmp
import matplotlib as mpl
import matplotlib.pyplot as plt

# from bds.bounds_utils import find_equivalence_classes
import numpy as np
import pandas as pd
import ray
from contexttimer import Timer
from gmpy2 import mpfr, mpz

# from bds.bb import BranchAndBoundNaive, BranchAndBoundV1, BranchAndBoundV0, BranchAndBoundV2
from logzero import logger
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model
from pylab import axes, boxplot, figure, legend, plot, savefig, setp, show, xlim, ylim
from scipy.sparse import random
from sklearn import datasets

from bds.bb import get_ground_truth_count
from bds.bbSampleTreeApproxCounting import BranchAndBoundNaive

# now we need to extract rules
from bds.fpgrowth import build_fptree, fpgrowth_on_tree, preprocess_transaction_list
from bds.gf2 import GF
from bds.meel import approx_mc2, approx_mc2_core
from bds.rule import Rule
from bds.sat.min_freq import construct_min_freq_program
from bds.sat.solver import construct_solver
from bds.utils import bin_array, bin_random, mpz_set_bits, randints, solutions_to_dict
from tests.utils import assert_close_mpfr, assert_dict_allclose

%cd ..



# from bds.cbb_v2 import *

logger.setLevel(logging.DEBUG)

num_pts = 500

show_progres = True
rand_seed = 1234

In [ ]:
# Load the CSV file using Pandas
data = pd.read_csv("data/compas_test-binary.csv")

# Print the data
data.head()

In [ ]:
# read labels
f = open("data/compas_test.label")
labels = []
for row in f.readlines():
    labels.append(list(map(int, row.split(" ")[1:])))

In [ ]:
y = np.array(labels[1], dtype=bool)

In [ ]:
X = data.to_numpy()

In [ ]:
assert len(y) == X.shape[0]

In [ ]:
# now we need to extract rules
X_bag = [{j for j, x in enumerate(row) if x} for row in X]

In [ ]:
lmbd = 0.1
min_support = lmbd * X.shape[0] / 2

In [ ]:
min_support

In [ ]:
def generate_random_sparse_binary_matrix(rows, cols, density):
    # Generate a random binary matrix with the specified density (percentage of non-zero elements)
    random_matrix = random(rows, cols, density=density, format="csr", dtype=np.float64)
    random_matrix.data = np.array(
        random_matrix.data >= 0.5, dtype=np.int64
    )  # Threshold to get binary values
    return random_matrix


class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print("%s=%i" % (v, self.Value(v)), end=" ")
        print()

    def solution_count(self):
        return self.__solution_count


def compute_truthtable_v2(X, itemset):
    """poorly optimized function"""
    support_list = []
    for i in range(X.shape[0]):
        if sum([X[i][j] for j in itemset]) == len(itemset):
            support_list.append(i)

    truthtable = mpz_set_bits(mpz(), support_list)

    return truthtable, support_list


def solve_sat_with_min_max(n, n_positives, n_negatives, N, P, lambd, epsilon):
    model = cp_model.CpModel()
    #
    m = len(P[0])
    #
    # Constraints for min function
    x = [model.NewBoolVar(f"x{i}") for i in range(m)]
    zFP = [
        model.NewBoolVar(f"FP{i}") for i in range(n_negatives)
    ]  # Auxiliary variables z_i with lower bound 0 and upper bound 1
    zFN = [model.NewBoolVar(f"FN{i}") for i in range(n_positives)]
    #
    zFPpre = [
        model.NewBoolVar(f"preFP{i}") for i in range(n_negatives)
    ]  # Auxiliary variables z_i with lower bound 0 and upper bound 1
    zFNpre = [model.NewBoolVar(f"preFN{i}") for i in range(n_positives)]
    #
    #
    sum_var_positive = model.NewIntVar(0, n_positives, "sumPos")
    sum_var_negative = model.NewIntVar(0, n_negatives, "sumNeg")
    sum_var_x = model.NewIntVar(0, m, "sumx")
    target = model.NewIntVar(0, n, "target")
    #
    #
    for i in range(n_negatives):
        model.Add(zFPpre[i] == sum([N[i][j] * x[j] for j in range(m)]))
        model.AddMinEquality(zFP[i], [zFPpre[i], 1])
    #
    # for i in range(n_negatives):
    #    model.AddMinEquality(zFP[i], [ sum([N[i][j] * x[j] for j in range(m)]) , 1]) # should be either 0 if the innter product is 0 or 1
    #
    #
    #
    for i in range(n_positives):
        model.Add(zFNpre[i] == sum([P[i][j] * x[j] for j in range(m)]))
        model.AddMaxEquality(zFN[i], [1 - zFNpre[i], 0])  # also 0 or 1

    model.Add(sum_var_positive == sum(zFN))
    #
    model.Add(sum_var_negative == sum(zFP))
    #
    model.Add(sum_var_x == sum(x))
    #
    model.Add(
        sum_var_positive + sum_var_negative + lambd * n * sum_var_x <= epsilon * n
    )
    #
    #
    # Create a solver
    # solver = cp_model.CpSolver()
    #
    # Find all feasible solutions
    # count = 0
    # while solver.NextSolution():
    #    count += 1
    #    print(f"Feasible Solution {count}:")
    #    for i in range(1, 6):
    #        print(f"x{i} = {solver.Value(x[i-1])}")

    solver = cp_model.CpSolver()
    # solution_printer = VarArraySolutionPrinter([x])
    ## Enumerate all solutions.
    # solver.parameters.enumerate_all_solutions = True
    # Solve.
    status = solver.Solve(model)
    # solver.SearchForAllSolutions(model, solution_printer)
    # count = 0

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("printing solutions... ")
        solution_printer = VarArraySolutionPrinter([x[i] for i in range(len(x))])
        ## Enumerate all solutions.
        solver.parameters.enumerate_all_solutions = True
        # Solve..
        status = solver.Solve(model, solution_printer)
        # print('x = %i' % solver.Value(x[0]))
    else:
        print("No solution found.")

    return model

In [ ]:
def custom_print(*args, **kwargs):
    # You can log the output to a file or ignore it altogether
    pass


# Redefine the print function inside your function
original_print = print
print = custom_print

In [ ]:
def compute_truthtable(X, itemset):
    """poorly optimized function"""
    support_list = []
    for i in range(X.shape[0]):
        if sum([X[i][j] for j in itemset]) == len(itemset):
            support_list.append(i)

    truthtable = mpz_set_bits(mpz(), support_list)

    return truthtable

In [ ]:
all_n_rules = []
lmbd = 1
ub = 3
rand_seed = 1234
n_reps = 1

delta = 0.8
eps = 0.8

show_progres = True

# all_averages_times_MC = []
# all_averages_times_GT = []


# all_averages_counts_MC = []
# all_averages_counts_GT = []

# all_averages_times_SAT = []

# for min_support in [ 50 ][::-1]:

for min_support in [30][::-1]:
    ordered_input_data = preprocess_transaction_list(X_bag, min_support)
    tree = build_fptree(ordered_input_data)
    frequent_itemsets = set(fpgrowth_on_tree(tree, set(), min_support))

    all_n_rules.append(len(frequent_itemsets))

    # Now create rules
    rules = []
    for i, itemset in enumerate(frequent_itemsets):
        tt = compute_truthtable(X, itemset)
        this_Rule = Rule(
            id=i + 1, name="rule_" + str(i), cardinality=len(itemset), truthtable=tt
        )
        # we need the truthtable for this itemset
        rules.append(this_Rule)

    pickle.dump(rules, open("rules/rules_compas_05_" + str(len(rules)), "wb"))

    # approxMC
    all_times_MC = []
    all_counts_MC = []

    all_times_GT = []
    all_counts_GT = []

    all_times_SAT = []

    for _ in range(n_reps):
        t = time.time()
        test_cnt = approx_mc2(
            rules,
            y,
            lmbd=lmbd,
            ub=ub,
            delta=delta,
            eps=eps,
            rand_seed=rand_seed,
            show_progress=show_progres,
            parallel=False,
            log_level=logging.WARN,
        )

        elapsed = time.time() - t
        all_times_MC.append(elapsed)
        all_counts_MC.append(test_cnt)

        # complete enumeration
        t = time.time()
        ref_count = get_ground_truth_count(rules, y, lmbd, ub)
        elapsed = time.time() - t
        all_times_GT.append(elapsed)
        all_counts_GT.append(ref_count)

        # sat
        PD = X[y]
        ND = X[~y]

        N = np.zeros((ND.shape[0], len(frequent_itemsets)))
        P = np.zeros((PD.shape[0], len(frequent_itemsets)))

        # create map
        positive_counter = 0
        negative_counter = 0
        map_positive = dict()
        map_negative = dict()

        for i in range(X.shape[0]):
            if y[i]:
                map_positive[i] = positive_counter
                positive_counter += 1
            else:
                map_negative[i] = negative_counter
                negative_counter += 1

        for i, itemset in enumerate(frequent_itemsets):
            tt, support_list = compute_truthtable_v2(X, itemset)
            # this_Rule = Rule(id = i+1, name = "rule_" + str(i), cardinality=len(itemset), truthtable=tt)
            # we need the truthtable for this itemset
            # rules.append(this_Rule)

            for example in support_list:
                if y[example]:
                    P[map_positive[example], i] = 1
                else:
                    N[map_negative[example], i] = 1

        # record time
        t = time.time()
        n = X.shape[0]
        rows_positive = P.shape[0]
        rows_negative = N.shape[0]
        solutions = solve_sat_with_min_max(
            n, rows_positive, rows_negative, N, P, lambd=lmbd, epsilon=ub
        )  # solve_integer_program(n, m, N, P, lmbd, ub)
        elapsed = time.time() - t
        all_times_SAT.append(elapsed)

    all_averages_times_MC.append(np.mean(all_times_MC))
    all_averages_times_GT.append(np.mean(all_times_GT))

    all_averages_counts_MC.append(np.mean(all_counts_MC))
    all_averages_counts_GT.append(np.mean(all_counts_GT))

    all_averages_times_SAT.append(np.mean(all_times_SAT))


pickle.dump(
    all_averages_times_MC,
    open("all_averages_times_MC_withSAT_2_" + str(len(rules)), "wb"),
)
pickle.dump(
    all_averages_times_GT,
    open("all_averages_times_GT_withSAT_2_" + str(len(rules)), "wb"),
)
pickle.dump(
    all_averages_counts_MC,
    open("all_averages_counts_MC_withSAT_2_" + str(len(rules)), "wb"),
)
pickle.dump(
    all_averages_counts_GT,
    open("all_averages_counts_GT_withSAT_2_" + str(len(rules)), "wb"),
)
pickle.dump(
    all_averages_times_SAT,
    open("all_averages_times_SAT_withSAT_2_" + str(len(rules)), "wb"),
)

# sampleTree
# bb = BranchAndBoundNaive(rules, ub, y, lmbd, l=4, k=150)
# Z = bb.runST()

In [ ]:
def move_last_to_first(lst):
    if len(lst) > 0:
        last_element = lst.pop()  # Remove the last element
        lst.insert(0, last_element)  # Insert the last element at the beginning

In [ ]:
all_averages_times_MC

In [ ]:
all_averages_times_GT

In [ ]:
move_last_to_first(all_averages_times_MC)
move_last_to_first(all_averages_times_GT)

In [ ]:
plt.style.use(["seaborn-notebook", "seaborn-darkgrid"])
mpl.rcParams["axes.facecolor"] = "white"
mpl.rcParams["grid.color"] = "lightgray"
mpl.rcParams["mathtext.default"] = "regular"

plt.figure(figsize=((5.5, 3.5)))
# Plot the first data on the first subplot
# ax1.plot([61 , 77, 104], y1, color='red')
# ax1.set_title('Plot 1')

x = [25, 32, 39, 61, 104]

plt.xlim(x[0] - 2, x[-1] + 2)

# print(times_MC)

plt.plot(
    x,
    times_by_rules,
    marker="X",
    color="firebrick",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)
# this is below

plt.plot(
    x,
    all_averages_times_MC[::-1],
    marker="o",
    color="orange",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)

plt.plot(
    x,
    all_averages_times_GT[::-1],
    marker="v",
    color="purple",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)

plt.plot(
    x,
    all_averages_times_SAT[::-1],
    marker="s",
    color="blue",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)

# ax1.legend(fontsize = 16)

# ax1.set_title('Runtime VS # Rules ( l = ' + str(l) + " k = " + str(k) + " )" , fontsize = 16)
plt.title("Runtime (s) by # Rules", fontsize=21)

plt.xlabel(" # Rules ", fontsize=21)

plt.ylabel(" Runtime (s) ", fontsize=21)

plt.yscale("log")  # Set y-axis to logarithmic scale
plt.xticks(fontsize=21)  # Change the fontsize to your desired value
plt.yticks(fontsize=21)

(hB,) = plot(
    [1, 1],
    "firebrick",
    marker="X",
    linewidth=3,
    markersize=10,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.8,
)
(hHP,) = plot(
    [1, 1],
    "orange",
    marker="o",
    linewidth=3,
    markersize=10,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.8,
)
(hS,) = plot(
    [1, 1],
    "purple",
    marker="v",
    linewidth=3,
    markersize=10,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.8,
)
#   hD, = plot([1,1],'salmon' , marker='D' ,  linewidth=3  , markersize = 10 , markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
#   hF, = plot([1,1], 'brown' , marker='P' ,  linewidth=3  , markersize = 10 , markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
# legend(( hB, hHP, hS,), ( "SampleTree-RS" , "ApproxMC-RS",   "FullBB-RS") ,loc='upper center', bbox_to_anchor=(-1, 1.25 ), ncol=3, fancybox=False, shadow=False, borderaxespad=0, fontsize = 14 )
# legend(( hB, hHP, hS, hD, hF), ( "Viterbi" , "Dijkstra",   "DijkstraBound", "BidirectionalDijkstra" , "BidirectionalDijkstraBound" ) ,loc='upper center', bbox_to_anchor=(0.5, 1.21 ), ncol=3, fancybox=True, shadow=True, borderaxespad=0 )
hB.set_visible(False)
hS.set_visible(False)
hHP.set_visible(False)

# Adjust spacing between subplots
plt.tight_layout()


plt.savefig("compas_runtime_vs_n_rules_with_SAT" + "_log.pdf")

# Display the plots
plt.show()

In [ ]:
plt.style.use(["seaborn-notebook", "seaborn-darkgrid"])
mpl.rcParams["axes.facecolor"] = "white"
mpl.rcParams["grid.color"] = "lightgray"
mpl.rcParams["mathtext.default"] = "regular"

plt.figure(figsize=((5.5, 3.5)))
# Plot the first data on the first subplot
# ax1.plot([61 , 77, 104], y1, color='red')
# ax1.set_title('Plot 1')

x = [25, 32, 39, 61, 104]

plt.xlim(x[0] - 2, x[-1] + 2)

# print(times_MC)

plt.plot(
    x,
    times_by_rules,
    marker="X",
    color="firebrick",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)
# this is below

plt.plot(
    x,
    all_averages_times_MC[::-1],
    marker="o",
    color="orange",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)

plt.plot(
    x,
    all_averages_times_GT[::-1],
    marker="v",
    color="purple",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)

plt.plot(
    x,
    all_averages_times_SAT[::-1],
    marker="s",
    color="blue",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)

# ax1.legend(fontsize = 16)

# ax1.set_title('Runtime VS # Rules ( l = ' + str(l) + " k = " + str(k) + " )" , fontsize = 16)
plt.title("Runtime (s) by # Rules", fontsize=21)

plt.xlabel(" # Rules ", fontsize=21)

plt.ylabel(" Runtime (s) ", fontsize=21)

# plt.yscale('log')  # Set y-axis to logarithmic scale

plt.xticks(fontsize=21)  # Change the fontsize to your desired value
plt.yticks(fontsize=21)

(hB,) = plot(
    [1, 1],
    "firebrick",
    marker="X",
    linewidth=3,
    markersize=10,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.8,
)
(hHP,) = plot(
    [1, 1],
    "orange",
    marker="o",
    linewidth=3,
    markersize=10,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.8,
)
(hS,) = plot(
    [1, 1],
    "purple",
    marker="v",
    linewidth=3,
    markersize=10,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.8,
)
#   hD, = plot([1,1],'salmon' , marker='D' ,  linewidth=3  , markersize = 10 , markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
#   hF, = plot([1,1], 'brown' , marker='P' ,  linewidth=3  , markersize = 10 , markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
# legend(( hB, hHP, hS,), ( "SampleTree-RS" , "ApproxMC-RS",   "FullBB-RS") ,loc='upper center', bbox_to_anchor=(-1, 1.25 ), ncol=3, fancybox=False, shadow=False, borderaxespad=0, fontsize = 14 )
# legend(( hB, hHP, hS, hD, hF), ( "Viterbi" , "Dijkstra",   "DijkstraBound", "BidirectionalDijkstra" , "BidirectionalDijkstraBound" ) ,loc='upper center', bbox_to_anchor=(0.5, 1.21 ), ncol=3, fancybox=True, shadow=True, borderaxespad=0 )
hB.set_visible(False)
hS.set_visible(False)
hHP.set_visible(False)

# Adjust spacing between subplots
plt.tight_layout()


plt.savefig("compas_runtime_vs_n_rules_with_SAT" + ".pdf")

# Display the plots
plt.show()

In [ ]:
data.shape

#### Now with mushroom dataset


In [ ]:
# Load the CSV file using Pandas
dataset = "mushrooms"
data = pd.read_csv("data/mushrooms.csv")

sample_size = 1000
data = data.sample(n=sample_size)


# Print the data
data.head()

In [ ]:
y = data["class"]
data = data.drop("class", axis=1)
data = pd.get_dummies(data).astype("int")
X = data.to_numpy()
X

In [ ]:
y = np.array(y, dtype=bool)
assert len(y) == X.shape[0]

In [ ]:
X.shape

In [ ]:
def compute_truthtable(X, itemset):
    """poorly optimized function"""
    support_list = []
    for i in range(X.shape[0]):
        if sum([X[i][j] for j in itemset]) == len(itemset):
            support_list.append(i)

    truthtable = mpz_set_bits(mpz(), support_list)

    return truthtable


X_bag = [{j for j, x in enumerate(row) if x} for row in X]


all_n_rules = []
lmbd = 1
ub = 2
rand_seed = 1234
n_reps = 1

delta = 0.8
eps = 0.8

show_progres = True

all_averages_times_MC = []
all_averages_times_GT = []

all_averages_counts_MC = []
all_averages_counts_GT = []

all_averages_times_SAT = []

In [ ]:
for min_support in [545]:
    ordered_input_data = preprocess_transaction_list(X_bag, min_support)
    tree = build_fptree(ordered_input_data)
    frequent_itemsets = set(fpgrowth_on_tree(tree, set(), min_support))

In [ ]:
len(frequent_itemsets)

In [ ]:
for min_support in [545]:
    ordered_input_data = preprocess_transaction_list(X_bag, min_support)
    tree = build_fptree(ordered_input_data)
    frequent_itemsets = set(fpgrowth_on_tree(tree, set(), min_support))

    all_n_rules.append(len(frequent_itemsets))

    # Now create rules
    rules = []
    for i, itemset in enumerate(frequent_itemsets):
        tt = compute_truthtable(X, itemset)
        this_Rule = Rule(
            id=i + 1, name="rule_" + str(i), cardinality=len(itemset), truthtable=tt
        )
        # we need the truthtable for this itemset
        rules.append(this_Rule)

    pickle.dump(rules, open("rules/rules_mushroom_05_" + str(len(rules)), "wb"))

    # approxMC
    all_times_MC = []
    all_counts_MC = []

    all_times_GT = []
    all_counts_GT = []

    all_times_SAT = []

    for _ in range(n_reps):
        t = time.time()
        test_cnt = approx_mc2(
            rules,
            y,
            lmbd=lmbd,
            ub=ub,
            delta=delta,
            eps=eps,
            rand_seed=rand_seed,
            show_progress=show_progres,
            parallel=False,
            log_level=logging.WARN,
        )

        elapsed = time.time() - t
        all_times_MC.append(elapsed)
        all_counts_MC.append(test_cnt)

        # complete enumeration
        t = time.time()
        ref_count = get_ground_truth_count(rules, y, lmbd, ub)
        elapsed = time.time() - t
        all_times_GT.append(elapsed)
        all_counts_GT.append(ref_count)

        # sat
        PD = X[y]
        ND = X[~y]

        N = np.zeros((ND.shape[0], len(frequent_itemsets)))
        P = np.zeros((PD.shape[0], len(frequent_itemsets)))

        # create map
        positive_counter = 0
        negative_counter = 0
        map_positive = dict()
        map_negative = dict()

        for i in range(X.shape[0]):
            if y[i]:
                map_positive[i] = positive_counter
                positive_counter += 1
            else:
                map_negative[i] = negative_counter
                negative_counter += 1

        for i, itemset in enumerate(frequent_itemsets):
            tt, support_list = compute_truthtable_v2(X, itemset)
            # this_Rule = Rule(id = i+1, name = "rule_" + str(i), cardinality=len(itemset), truthtable=tt)
            # we need the truthtable for this itemset
            # rules.append(this_Rule)

            for example in support_list:
                if y[example]:
                    P[map_positive[example], i] = 1
                else:
                    N[map_negative[example], i] = 1

        # record time
        t = time.time()
        n = X.shape[0]
        rows_positive = P.shape[0]
        rows_negative = N.shape[0]
        solutions = solve_sat_with_min_max(
            n, rows_positive, rows_negative, N, P, lambd=lmbd, epsilon=ub
        )  # solve_integer_program(n, m, N, P, lmbd, ub)
        elapsed = time.time() - t
        all_times_SAT.append(elapsed)

    all_averages_times_MC.append(np.mean(all_times_MC))
    all_averages_times_GT.append(np.mean(all_times_GT))

    all_averages_counts_MC.append(np.mean(all_counts_MC))
    all_averages_counts_GT.append(np.mean(all_counts_GT))

    all_averages_times_SAT.append(np.mean(all_times_SAT))


pickle.dump(
    all_averages_times_MC,
    open("all_averages_times_MC_withSAT_2_mushroom_" + str(len(rules)), "wb"),
)
pickle.dump(
    all_averages_times_GT,
    open("all_averages_times_GT_withSAT_2_mushroom_" + str(len(rules)), "wb"),
)
pickle.dump(
    all_averages_counts_MC,
    open("all_averages_counts_MC_withSAT_2_mushroom_" + str(len(rules)), "wb"),
)
pickle.dump(
    all_averages_counts_GT,
    open("all_averages_counts_GT_withSAT_2_mushroom_" + str(len(rules)), "wb"),
)
pickle.dump(
    all_averages_times_SAT,
    open("all_averages_times_SAT_withSAT_2_mushroom_" + str(len(rules)), "wb"),
)

In [ ]:
all_averages_times_SAT

In [ ]:
all_averages_times_MC

In [ ]:
results_sampleTree_counts = dict()
results_sampleTree_times = dict()
lmbd = 0.1
ub = 0.3
rand_seed = 1234
n_reps = 1

delta = 0.8
eps = 0.8

all_n_rules = [25, 33, 65, 89, 95]


for l in [16]:
    for k in [
        506
    ]:  # [100, 225, 506, 1139, 2562, 2562, 5764, 12969, 29180]: #[100, 225, 506, 1139, 2562, 2562, 5764]:
        times_by_rules = []
        counts_by_rules = []

        for n_rules in all_n_rules:
            print(n_rules)

            rules = read_pickle("rules/rules_mushroom_05_" + str(n_rules))[0]

            all_times = []
            all_counts = []

            for _ in range(10):
                ts = time.time()
                bb = BranchAndBoundNaive(rules, ub, y, lmbd, l=l, k=k)
                Z = bb.runST()
                elapsed = time.time() - ts
                all_times.append(elapsed)
                all_counts.append(Z)

            times_by_rules.append(np.mean(all_times))
            counts_by_rules.append(np.mean(all_counts))

        pickle.dump(
            times_by_rules,
            open(
                "mushroom_1000_times_by_rules_5_"
                + str(len(rules))
                + "_"
                + str(l)
                + "_"
                + str(k),
                "wb",
            ),
        )
        pickle.dump(
            counts_by_rules,
            open(
                "mushroom_1000_counts_by_rules_5_"
                + str(len(rules))
                + "_"
                + str(l)
                + "_"
                + str(k),
                "wb",
            ),
        )

In [ ]:
plt.style.use(["seaborn-notebook", "seaborn-darkgrid"])
mpl.rcParams["axes.facecolor"] = "white"
mpl.rcParams["grid.color"] = "lightgray"
mpl.rcParams["mathtext.default"] = "regular"

plt.figure(figsize=((5.5, 3.5)))
# Plot the first data on the first subplot
# ax1.plot([61 , 77, 104], y1, color='red')
# ax1.set_title('Plot 1')

x = [25, 33, 65, 89, 95]

plt.xlim(x[0] - 2, x[-1] + 2)

# print(times_MC)

plt.plot(
    x,
    times_by_rules,
    marker="X",
    color="firebrick",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)
# this is below

plt.plot(
    x,
    all_averages_times_MC,
    marker="o",
    color="orange",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)

plt.plot(
    x,
    all_averages_times_GT,
    marker="v",
    color="purple",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)

plt.plot(
    x,
    all_averages_times_SAT,
    marker="s",
    color="blue",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)

# ax1.legend(fontsize = 16)

# ax1.set_title('Runtime VS # Rules ( l = ' + str(l) + " k = " + str(k) + " )" , fontsize = 16)
plt.title("Runtime (s) by # Rules", fontsize=21)

plt.xlabel(" # Rules ", fontsize=21)

plt.ylabel(" Runtime (s) ", fontsize=21)

plt.yscale("log")  # Set y-axis to logarithmic scale
plt.xticks(fontsize=21)  # Change the fontsize to your desired value
plt.yticks(fontsize=21)

(hB,) = plot(
    [1, 1],
    "firebrick",
    marker="X",
    linewidth=3,
    markersize=10,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.8,
)
(hHP,) = plot(
    [1, 1],
    "orange",
    marker="o",
    linewidth=3,
    markersize=10,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.8,
)
(hS,) = plot(
    [1, 1],
    "purple",
    marker="v",
    linewidth=3,
    markersize=10,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.8,
)
#   hD, = plot([1,1],'salmon' , marker='D' ,  linewidth=3  , markersize = 10 , markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
#   hF, = plot([1,1], 'brown' , marker='P' ,  linewidth=3  , markersize = 10 , markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
# legend(( hB, hHP, hS,), ( "SampleTree-RS" , "ApproxMC-RS",   "FullBB-RS") ,loc='upper center', bbox_to_anchor=(-1, 1.25 ), ncol=3, fancybox=False, shadow=False, borderaxespad=0, fontsize = 14 )
# legend(( hB, hHP, hS, hD, hF), ( "Viterbi" , "Dijkstra",   "DijkstraBound", "BidirectionalDijkstra" , "BidirectionalDijkstraBound" ) ,loc='upper center', bbox_to_anchor=(0.5, 1.21 ), ncol=3, fancybox=True, shadow=True, borderaxespad=0 )
hB.set_visible(False)
hS.set_visible(False)
hHP.set_visible(False)

# Adjust spacing between subplots
plt.tight_layout()


plt.savefig("mushroom_1000_runtime_vs_n_rules_with_SAT" + "_log.pdf")

# Display the plots
plt.show()

In [ ]:
plt.style.use(["seaborn-notebook", "seaborn-darkgrid"])
mpl.rcParams["axes.facecolor"] = "white"
mpl.rcParams["grid.color"] = "lightgray"
mpl.rcParams["mathtext.default"] = "regular"

plt.figure(figsize=((5.5, 3.5)))
# Plot the first data on the first subplot
# ax1.plot([61 , 77, 104], y1, color='red')
# ax1.set_title('Plot 1')

x = [25, 32, 39, 61, 104]

plt.xlim(x[0] - 2, x[-1] + 2)

# print(times_MC)

plt.plot(
    x,
    times_by_rules,
    marker="X",
    color="firebrick",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)
# this is below

plt.plot(
    x,
    all_averages_times_MC,
    marker="o",
    color="orange",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)

plt.plot(
    x,
    all_averages_times_GT,
    marker="v",
    color="purple",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)

plt.plot(
    x,
    all_averages_times_SAT,
    marker="s",
    color="blue",
    linewidth=5,
    markersize=12,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.5,
)

# ax1.legend(fontsize = 16)

# ax1.set_title('Runtime VS # Rules ( l = ' + str(l) + " k = " + str(k) + " )" , fontsize = 16)
plt.title("Runtime (s) by # Rules", fontsize=21)

plt.xlabel(" # Rules ", fontsize=21)

plt.ylabel(" Runtime (s) ", fontsize=21)

# plt.yscale('log')  # Set y-axis to logarithmic scale

plt.xticks(fontsize=21)  # Change the fontsize to your desired value
plt.yticks(fontsize=21)

(hB,) = plot(
    [1, 1],
    "firebrick",
    marker="X",
    linewidth=3,
    markersize=10,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.8,
)
(hHP,) = plot(
    [1, 1],
    "orange",
    marker="o",
    linewidth=3,
    markersize=10,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.8,
)
(hS,) = plot(
    [1, 1],
    "purple",
    marker="v",
    linewidth=3,
    markersize=10,
    markeredgewidth=0.5,
    markeredgecolor="grey",
    alpha=0.8,
)
#   hD, = plot([1,1],'salmon' , marker='D' ,  linewidth=3  , markersize = 10 , markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
#   hF, = plot([1,1], 'brown' , marker='P' ,  linewidth=3  , markersize = 10 , markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
# legend(( hB, hHP, hS,), ( "SampleTree-RS" , "ApproxMC-RS",   "FullBB-RS") ,loc='upper center', bbox_to_anchor=(-1, 1.25 ), ncol=3, fancybox=False, shadow=False, borderaxespad=0, fontsize = 14 )
# legend(( hB, hHP, hS, hD, hF), ( "Viterbi" , "Dijkstra",   "DijkstraBound", "BidirectionalDijkstra" , "BidirectionalDijkstraBound" ) ,loc='upper center', bbox_to_anchor=(0.5, 1.21 ), ncol=3, fancybox=True, shadow=True, borderaxespad=0 )
hB.set_visible(False)
hS.set_visible(False)
hHP.set_visible(False)

# Adjust spacing between subplots
plt.tight_layout()


plt.savefig("mushroom_1000_runtime_vs_n_rules_with_SAT" + ".pdf")

# Display the plots
plt.show()

In [ ]:
all_times_SAT

In [ ]:
times_by_rules

In [ ]:
all_n_rules = [25, 32, 39, 61, 104]

In [ ]:
pickle.dump(rules, open("rules_compas_" + str(len(rules)), "wb"))

In [ ]:
times_by_rules

In [ ]:
results_sampleTree_counts = dict()
results_sampleTree_times = dict()
lmbd = 0.1
ub = 0.3
rand_seed = 1234
n_reps = 1

delta = 0.8
eps = 0.8

all_n_rules = [25, 32, 39, 61, 104]


for l in [16]:
    for k in [
        506
    ]:  # [100, 225, 506, 1139, 2562, 2562, 5764, 12969, 29180]: #[100, 225, 506, 1139, 2562, 2562, 5764]:
        times_by_rules = []
        counts_by_rules = []

        for n_rules in all_n_rules:
            print(n_rules)

            rules = read_pickle("rules/rules_compas_04_" + str(n_rules))[0]

            all_times = []
            all_counts = []

            for _ in range(10):
                ts = time.time()
                bb = BranchAndBoundNaive(rules, ub, y, lmbd, l=l, k=k)
                Z = bb.runST()
                elapsed = time.time() - ts
                all_times.append(elapsed)
                all_counts.append(Z)

            times_by_rules.append(np.mean(all_times))
            counts_by_rules.append(np.mean(all_counts))

        pickle.dump(
            times_by_rules,
            open(
                "times_by_rules_4_" + str(len(rules)) + "_" + str(l) + "_" + str(k),
                "wb",
            ),
        )
        pickle.dump(
            counts_by_rules,
            open(
                "counts_by_rules_4_" + str(len(rules)) + "_" + str(l) + "_" + str(k),
                "wb",
            ),
        )

In [ ]:
def read_pickle(filepath_y):
    objects = []
    with open(filepath_y, "rb") as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break

    return objects

In [ ]:
5764 * 1.5**2 * 1.5**2

In [ ]:
for l in [1, 2, 4, 8]:
    for k in [100, 225, 506, 1139, 2562, 2562, 3000, 4000, 5764, 12969, 29180]:
        if k > 5765 and l > 2:
            continue

        times = read_pickle("times_by_rules_" + str(104) + "_" + str(l) + "_" + str(k))[
            0
        ]
        counts = read_pickle(
            "counts_by_rules_" + str(104) + "_" + str(l) + "_" + str(k)
        )[0]

        times_MC = []
        counts_MC = []

        times_GT = []
        counts_GT = []

        for n_rules in [61, 77, 104]:
            this_times_MC = read_pickle("all_averages_times_MC_" + str(n_rules))[0]

            times_MC.append(this_times_MC[0])

            this_cnts_MC = read_pickle("all_averages_counts_MC_" + str(n_rules))[0]

            counts_MC.append(this_cnts_MC[0])

            this_times_GT = read_pickle("all_averages_times_GT_" + str(n_rules))[0]

            times_GT.append(this_times_GT[0])

            this_cnts_GT = read_pickle("all_averages_counts_GT_" + str(n_rules))[0]

            counts_GT.append(this_cnts_MC[0])

        # plt.style.use(['seaborn-notebook', 'seaborn-darkgrid'])
        mpl.rcParams["axes.facecolor"] = "white"
        mpl.rcParams["grid.color"] = "lightgray"
        mpl.rcParams["mathtext.default"] = "regular"
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
        # Plot the first data on the first subplot
        # ax1.plot([61 , 77, 104], y1, color='red')
        # ax1.set_title('Plot 1')

        x = [61, 77, 104]

        # print(times_MC)

        ax1.plot(
            x,
            times,
            marker="X",
            color="firebrick",
            label="SampleTree-RS",
            linewidth=5,
            markersize=12,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.5,
        )

        ax1.plot(
            x,
            times_MC,
            marker="o",
            color="orange",
            label="ApproxMC-RS",
            linewidth=5,
            markersize=12,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.5,
        )

        # plt.plot(alllent_sequence, all_naive_se_times_ha, marker='P', color = "green" , linewidth=3 , markersize = 10 ,markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
        ax1.plot(
            x,
            times_GT,
            marker="v",
            color="purple",
            label="FullBB-RS",
            linewidth=5,
            markersize=12,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.5,
        )

        ax1.legend(fontsize=16)

        ax1.set_title(
            "Runtime VS # Rules ( l = " + str(l) + " k = " + str(k) + " )", fontsize=16
        )

        ax1.set_xlabel(" # Rules ", fontsize=14)

        ax1.set_ylabel(" Runtime (s) ", fontsize=14)

        ax1.set_yscale("log")  # Set y-axis to logarithmic scale

        ax2.plot(
            x,
            counts,
            marker="X",
            color="firebrick",
            label="SampleTree-RS",
            linewidth=5,
            markersize=12,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.5,
        )

        ax2.plot(
            x,
            counts_MC,
            marker="o",
            color="orange",
            label="ApproxMC-RS",
            linewidth=5,
            markersize=12,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.5,
        )

        # plt.plot(alllent_sequence, all_naive_se_times_ha, marker='P', color = "green" , linewidth=3 , markersize = 10 ,markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
        ax2.plot(
            x,
            counts_GT,
            marker="v",
            color="purple",
            label="FullBB-RS",
            linewidth=5,
            markersize=12,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.5,
        )

        ax2.legend(fontsize=16)

        ax2.set_title(
            "Counts VS # Rules ( l = " + str(l) + " k = " + str(k) + " )", fontsize=16
        )

        ax2.set_xlabel(" # Rules ", fontsize=14)

        ax2.set_ylabel(" Counts", fontsize=14)

        ax2.set_yscale("log")  # Set y-axis to logarithmic scale

        (hB,) = plot(
            [1, 1],
            "firebrick",
            marker="X",
            linewidth=3,
            markersize=10,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.8,
        )
        (hHP,) = plot(
            [1, 1],
            "orange",
            marker="o",
            linewidth=3,
            markersize=10,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.8,
        )
        (hS,) = plot(
            [1, 1],
            "purple",
            marker="v",
            linewidth=3,
            markersize=10,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.8,
        )
        #   hD, = plot([1,1],'salmon' , marker='D' ,  linewidth=3  , markersize = 10 , markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
        #   hF, = plot([1,1], 'brown' , marker='P' ,  linewidth=3  , markersize = 10 , markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
        # legend(( hB, hHP, hS,), ( "SampleTree-RS" , "ApproxMC-RS",   "FullBB-RS") ,loc='upper center', bbox_to_anchor=(-1, 1.25 ), ncol=3, fancybox=False, shadow=False, borderaxespad=0, fontsize = 14 )
        # legend(( hB, hHP, hS, hD, hF), ( "Viterbi" , "Dijkstra",   "DijkstraBound", "BidirectionalDijkstra" , "BidirectionalDijkstraBound" ) ,loc='upper center', bbox_to_anchor=(0.5, 1.21 ), ncol=3, fancybox=True, shadow=True, borderaxespad=0 )
        hB.set_visible(False)
        hS.set_visible(False)
        hHP.set_visible(False)

        # Adjust spacing between subplots
        plt.tight_layout()

        plt.savefig("sample_tree_by_rules_" + str(l) + "_" + str(k) + "_log.pdf")

        # Display the plots
        plt.show()

In [ ]:
for l in [1, 4]:
    for k in [100, 225, 506, 1139, 2562, 5764]:
        # if k > 5765 and l > 2:
        #    continue

        times = read_pickle(
            "times_by_rules_2_" + str(312) + "_" + str(l) + "_" + str(k)
        )[0]
        counts = read_pickle(
            "counts_by_rules_2_" + str(312) + "_" + str(l) + "_" + str(k)
        )[0]

        times_MC = []
        counts_MC = []

        times_GT = []
        counts_GT = []

        # for n_rules in all_n_rules:

        times_MC = read_pickle("all_averages_times_MC_" + str(312))[0]

        # "times_MC.append(this_times_MC[0])

        cnts_MC = read_pickle("all_averages_counts_MC_" + str(312))[0]

        # counts_MC.append(this_cnts_MC[0])

        times_GT = read_pickle("all_averages_times_GT_" + str(312))[0]

        #   times_GT.append(this_times_GT[0])

        cnts_GT = read_pickle("all_averages_counts_GT_" + str(312))[0]

        # counts_GT.append(this_cnts_MC[0])

        # plt.style.use(['seaborn-notebook', 'seaborn-darkgrid'])
        mpl.rcParams["axes.facecolor"] = "white"
        mpl.rcParams["grid.color"] = "lightgray"
        mpl.rcParams["mathtext.default"] = "regular"
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
        # Plot the first data on the first subplot
        # ax1.plot([61 , 77, 104], y1, color='red')
        # ax1.set_title('Plot 1')

        all_n_rules = [61, 77, 104, 151, 262, 312]

        x = all_n_rules

        # print(times_MC)

        ax1.plot(
            x,
            times,
            marker="X",
            color="firebrick",
            label="SampleTree-RS",
            linewidth=5,
            markersize=12,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.5,
        )

        ax1.plot(
            x,
            times_MC,
            marker="o",
            color="orange",
            label="ApproxMC-RS",
            linewidth=5,
            markersize=12,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.5,
        )

        # plt.plot(alllent_sequence, all_naive_se_times_ha, marker='P', color = "green" , linewidth=3 , markersize = 10 ,markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
        ax1.plot(
            x,
            times_GT,
            marker="v",
            color="purple",
            label="FullBB-RS",
            linewidth=5,
            markersize=12,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.5,
        )

        ax1.legend(fontsize=16)

        ax1.set_title(
            "Runtime VS # Rules ( l = " + str(l) + " k = " + str(k) + " )", fontsize=16
        )

        ax1.set_xlabel(" # Rules ", fontsize=14)

        ax1.set_ylabel(" Runtime (s) ", fontsize=14)

        ax1.set_yscale("log")  # Set y-axis to logarithmic scale

        ax2.plot(
            x,
            counts,
            marker="X",
            color="firebrick",
            label="SampleTree-RS",
            linewidth=5,
            markersize=12,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.5,
        )

        ax2.plot(
            x,
            cnts_MC,
            marker="o",
            color="orange",
            label="ApproxMC-RS",
            linewidth=5,
            markersize=12,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.5,
        )

        # plt.plot(alllent_sequence, all_naive_se_times_ha, marker='P', color = "green" , linewidth=3 , markersize = 10 ,markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
        ax2.plot(
            x,
            cnts_GT,
            marker="v",
            color="purple",
            label="FullBB-RS",
            linewidth=5,
            markersize=12,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.5,
        )

        ax2.legend(fontsize=16)

        ax2.set_title(
            "Counts VS # Rules ( l = " + str(l) + " k = " + str(k) + " )", fontsize=16
        )

        ax2.set_xlabel(" # Rules ", fontsize=14)

        ax2.set_ylabel(" Counts", fontsize=14)

        ax2.set_yscale("log")  # Set y-axis to logarithmic scale

        (hB,) = plot(
            [1, 1],
            "firebrick",
            marker="X",
            linewidth=3,
            markersize=10,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.8,
        )
        (hHP,) = plot(
            [1, 1],
            "orange",
            marker="o",
            linewidth=3,
            markersize=10,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.8,
        )
        (hS,) = plot(
            [1, 1],
            "purple",
            marker="v",
            linewidth=3,
            markersize=10,
            markeredgewidth=0.5,
            markeredgecolor="grey",
            alpha=0.8,
        )
        #   hD, = plot([1,1],'salmon' , marker='D' ,  linewidth=3  , markersize = 10 , markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
        #   hF, = plot([1,1], 'brown' , marker='P' ,  linewidth=3  , markersize = 10 , markeredgewidth=0.5, markeredgecolor= "grey" , alpha = 0.8)
        # legend(( hB, hHP, hS,), ( "SampleTree-RS" , "ApproxMC-RS",   "FullBB-RS") ,loc='upper center', bbox_to_anchor=(-1, 1.25 ), ncol=3, fancybox=False, shadow=False, borderaxespad=0, fontsize = 14 )
        # legend(( hB, hHP, hS, hD, hF), ( "Viterbi" , "Dijkstra",   "DijkstraBound", "BidirectionalDijkstra" , "BidirectionalDijkstraBound" ) ,loc='upper center', bbox_to_anchor=(0.5, 1.21 ), ncol=3, fancybox=True, shadow=True, borderaxespad=0 )
        hB.set_visible(False)
        hS.set_visible(False)
        hHP.set_visible(False)

        # Adjust spacing between subplots
        plt.tight_layout()

        plt.savefig("sample_tree_by_rules_3_" + str(l) + "_" + str(k) + "_log.pdf")

        # Display the plots
        plt.show()

In [ ]:
logger.setLevel(logging.WARN)

num_pts = 1000

ub = 0.8
lmbd = 0.1
delta = 0.8
eps = 0.8
show_progres = True
rand_seed = 1234

ray.init(num_cpus=4)

In [ ]:
test_cnt = approx_mc2(
    rules,
    y,
    lmbd=lmbd,
    ub=ub,
    delta=delta,
    eps=eps,
    rand_seed=rand_seed,
    show_progress=show_progres,
    parallel=True,
    log_level=logging.WARN,
)

In [ ]:
ref_count = get_ground_truth_count(rules, y, lmbd, ub)